In [1]:
"""
Some quick plots for the isolated disk setup.
"""
import numpy as np
import h5py
import glob
from os.path import isfile, isdir
from os import mkdir

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sphMap import sphMap

In [2]:
def units():
    """ Load/derive the unit system. """

    # get a snapshot
    snap = glob.glob(path + 'snap*.hdf5')[0]

    with h5py.File(snap,'r') as f:
        attrs = dict(f['Header'].attrs)

    # unit system
    u = {k:v for k,v in attrs.items() if 'Unit' in k}

    u['BoxSize'] = attrs['BoxSize']

    # derived units
    u['UnitTime_in_s']       = u['UnitLength_in_cm'] / u['UnitVelocity_in_cm_per_s']
    u['UnitDensity_in_cgs']  = u['UnitMass_in_g'] / u['UnitLength_in_cm']**3
    u['UnitPressure_in_cgs'] = u['UnitMass_in_g'] / u['UnitLength_in_cm'] / u['UnitTime_in_s']**2
    u['UnitEnergy_in_cgs']   = u['UnitMass_in_g'] * u['UnitLength_in_cm']**2 / u['UnitTime_in_s']**2
    u['UnitTemp_in_cgs']     = u['UnitEnergy_in_cgs'] / u['UnitMass_in_g']

    # helpful conversions
    sec_in_year = 3.155693e7
    Msun_in_g = 1.98892e33
    kpc_in_cm = 3.085680e21

    u['UnitTime_in_Gyr'] = u['UnitTime_in_s'] / sec_in_year / 1e9
    u['UnitMass_in_Msun'] = u['UnitMass_in_g'] / Msun_in_g
    u['UnitLength_in_kpc'] = u['UnitLength_in_cm'] / kpc_in_cm

    u['UnitDensity_in_Msun_kpc3'] = u['UnitDensity_in_cgs'] * (u['UnitMass_in_Msun']/u['UnitLength_in_kpc']**3)

    return u

def numpart_vs_time():
    """ Plot number of gas/stars/DM, versus time. """
    snaps = sorted(glob.glob(path + 'snap*.hdf5'))

    # load
    times = []
    numgas = []
    numdm = []
    numstars = []

    for snap in snaps:
        with h5py.File(snap,'r') as f:
            attrs = dict(f['Header'].attrs)

        times.append(attrs['Time'])
        numgas.append(attrs['NumPart_Total'][0])
        numdm.append(attrs['NumPart_Total'][1])
        numstars.append(attrs['NumPart_Total'][4])

    # convert times to Gyr
    u = units()

    times = np.array(times) * u['UnitTime_in_Gyr']

    # plot
    fig, ax = plt.subplots(figsize=(11,8))

    ax.set_xlabel('Time [ Gyr ]')
    ax.set_ylabel('Number of Gas Cells')
    ax.set_yscale('log')

    ax.plot(times, numgas, '-', lw=2.5, label='Gas')
    ax.plot(times, numstars, '-', lw=2.5, label='Stars')
    ax.plot(times, numdm, '-', lw=2.5, label='DM')

    ax.legend(loc='best')
    fig.savefig(savePath + 'numpart_vs_time.png')
    plt.close(fig)
    
def partmass_vs_time():
    """ Plot mass of gas/stars/DM, versus time. """
    snaps = sorted(glob.glob(path + 'snap*.hdf5'))

    # load
    times = []
    numgas = []
    numdm = []
    numstars = []
    gasMass = []
    starMass = []

    for snap in snaps:
        with h5py.File(snap,'r') as f:
            attrs = dict(f['Header'].attrs)
            partType0 = dict(f['PartType0'])
            gasMasses = partType0["Masses"]
            totalGasMass = np.sum(gasMasses)
            try:
                f['PartType4']
            except KeyError:
                continue
            else:
                partType4 = dict(f['PartType4'])
                starMasses = partType4["Masses"]
                totalStarMass = np.sum(starMasses)
                starMass.append(totalStarMass)

        times.append(attrs['Time'])
        numgas.append(attrs['NumPart_Total'][0])
        numdm.append(attrs['NumPart_Total'][1])
        numstars.append(attrs['NumPart_Total'][4])
        gasMass.append(totalGasMass)
    # convert times to Gyr
    u = units()

    times = np.array(times) * u['UnitTime_in_Gyr']

    # plot
    fig, ax = plt.subplots(figsize=(11,8))

    ax.set_xlabel('Time [ Gyr ]')
    ax.set_ylabel('Mass of Gas Cells [10e10 M_sun]')
    ax.set_yscale('log')

    ax.plot(times, gasMass, '-', lw=2.5, label='Gas Mass')
    #ax.plot(times, dmMass, '-', lw=2.5, label='DM Mass')
    ax.plot(times, starMass, '-', lw=2.5, label='Star Mass')

    ax.legend(loc='best')
    fig.savefig(savePath + 'partMass_vs_time.png')
    plt.close(fig)

def visualize_frames():
    """ Create individual images frames, to be combined into a movie, visualizing the gas. 
    Encode with:

    ffmpeg -f image2 -start_number 0 -i frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -crf 19 -an -threads 0 out.mp4

    """
    nbins = 500
    vmm = [4.0, 8.0] # log msun/kpc^2

    size_faceon = [70,130] # kpc, i.e. center of box
    size_edgeon = [85,115] # kpc, i.e. narrow for edge-on

    if not isdir('frames'):
        mkdir('frames')

    # get list of snapshots, and units
    snaps = sorted(glob.glob(path + 'snap*.hdf5'))
    u = units()

    # loop over each snapshot
    for i, snap in enumerate(snaps):
        # output file
        filename = savePath + 'frame_%03d.png' % i
        print(i)

        # frame already exists?
        if(overrideOldFrames == False):
            if isfile(filename):
                continue
                
        # load
        with h5py.File(snap,'r') as f:
            pos = f['PartType0']['Coordinates'][()]
            mass = f['PartType0']['Masses'][()]
            dens = f['PartType0']['Density'][()]
            volume = mass / dens

        # convert masses from code units to msun, positions to kpc
        pos *= u['UnitLength_in_kpc']
        mass *= u['UnitMass_in_Msun']

        if 0:
            # histogram face-on, and edge-on
            extent = [[0,u['BoxSize']],[0,u['BoxSize']]]

            h2d_faceon, _, _ = np.histogram2d(pos[:,0], pos[:,1], weights=mass, bins=nbins, range=extent)
            h2d_edgeon, _, _ = np.histogram2d(pos[:,0], pos[:,2], weights=mass, bins=nbins, range=extent)

        # new: use sphMap to generate projection images
        cell_radius = (volume * 3.0 / (4*np.pi))**(1.0/3.0) # assume spherical
        size = 2.5 * cell_radius # kpc, factor of 2.5 means slightly larger than cell diameter

        boxCen = [u['BoxSize']/2,u['BoxSize']/2,u['BoxSize']/2]
        boxSizeImg = [u['BoxSize'],u['BoxSize'],u['BoxSize']]

        # quant: put in zVelocity (1 nbr for each pos)
        # hist1 = sphMap(), 1  is img of quantity 
        h2d_faceon = sphMap(pos, size, mass, quant=None, axes=[0,1], boxSizeImg=boxSizeImg, 
                            boxSizeSim=0, boxCen=boxCen, nPixels=[nbins,nbins], ndims=3)

        h2d_edgeon = sphMap(pos, size, mass, quant=None, axes=[2,0], boxSizeImg=boxSizeImg, 
                            boxSizeSim=0, boxCen=boxCen, nPixels=[nbins,nbins], ndims=3)

        # normalize by pixel area: [msun per pixel] -> [msun/kpc^2]
        px_area = (boxSizeImg[0]/nbins)**2 # kpc^2

        h2d_faceon /= px_area
        h2d_edgeon /= px_area

        h2d_faceon[h2d_faceon > 0] = np.log10(h2d_faceon[h2d_faceon > 0])
        h2d_edgeon[h2d_edgeon > 0] = np.log10(h2d_edgeon[h2d_edgeon > 0])

        # plot
        fig, (ax_left, ax_right) = plt.subplots(nrows=1, ncols=2, figsize=(13,8), width_ratios=[0.63,0.37], dpi=300) # 

        ax_left.set_xlabel('x [kpc]')
        ax_left.set_ylabel('y [kpc]')
        ax_left.set_xlim(size_faceon)
        ax_left.set_ylim(size_faceon)

        extent = [0,u['BoxSize'],0,u['BoxSize']]
        im_left = ax_left.imshow(h2d_faceon, cmap='inferno', extent=extent, aspect=1.0, vmin=vmm[0], vmax=vmm[1])

        ax_right.set_xlabel('z [kpc]')
        ax_right.set_ylabel('x [kpc]')
        ax_right.set_xlim(size_edgeon)
        ax_right.set_ylim(size_faceon) # should match above

        im_right = ax_right.imshow(h2d_edgeon, cmap='inferno', extent=extent, aspect=1.0, vmin=vmm[0], vmax=vmm[1])

        # colorbar and finish plot
        cax = make_axes_locatable(ax_right).append_axes('right', size='10%', pad=0.1)
        cb = plt.colorbar(im_right, cax=cax)
        cb.ax.set_ylabel('Gas Surface Mass Density [ $\\rm{M_\odot kpc^{-2}}$ ]')

        fig.savefig(filename)
        plt.close(fig)

        
def visualize_frames_cluster(vmm):
    """ Create individual images frames, to be combined into a movie, visualizing the gas. 
    Encode with:

    ffmpeg -f image2 -start_number 0 -i frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -crf 19 -an -threads 0 out.mp4

    """
    nbins = 500
    #vmm = [-1, 2.6] # log msun/kpc^2

    size_faceon = [0,96] # kpc, i.e. center of box
    size_edgeon = [0,96] # kpc, i.e. narrow for edge-on

    if not isdir('frames'):
        mkdir('frames')

    # get list of snapshots, and units
    snaps = sorted(glob.glob(path + 'snap*.hdf5'))
    u = units()

    # loop over each snapshot
    for i, snap in enumerate(snaps):
        # output file
        filename = savePath + 'frame_%03d.png' % i
        print(i)

        # frame already exists?
        if(overrideOldFrames == False):
            if isfile(filename):
                continue
                
        # load
        with h5py.File(snap,'r') as f:
            pos = f['PartType0']['Coordinates'][()]
            mass = f['PartType0']['Masses'][()]
            dens = f['PartType0']['Density'][()]
            volume = mass / dens

        # convert masses from code units to msun, positions to pc
        pos *= u['UnitLength_in_kpc'] * 1000
        mass *= u['UnitMass_in_Msun']

        if 0:
            # histogram face-on, and edge-on
            extent = [[0,u['BoxSize']],[0,u['BoxSize']]]

            h2d_faceon, _, _ = np.histogram2d(pos[:,0], pos[:,1], weights=mass, bins=nbins, range=extent)
            h2d_edgeon, _, _ = np.histogram2d(pos[:,0], pos[:,2], weights=mass, bins=nbins, range=extent)

        # new: use sphMap to generate projection images
        cell_radius = (volume * 3.0 / (4*np.pi))**(1.0/3.0) # assume spherical
        size = 2.5 * cell_radius # kpc, factor of 2.5 means slightly larger than cell diameter

        boxCen = [u['BoxSize']/2,u['BoxSize']/2,u['BoxSize']/2]
        boxSizeImg = [u['BoxSize'],u['BoxSize'],u['BoxSize']]

        h2d_faceon = sphMap(pos, size, mass, quant=None, axes=[0,1], boxSizeImg=boxSizeImg, 
                            boxSizeSim=0, boxCen=boxCen, nPixels=[nbins,nbins], ndims=3)

        h2d_edgeon = sphMap(pos, size, mass, quant=None, axes=[2,0], boxSizeImg=boxSizeImg, 
                            boxSizeSim=0, boxCen=boxCen, nPixels=[nbins,nbins], ndims=3)

        # normalize by pixel area: [msun per pixel] -> [msun/kpc^2]
        px_area = (boxSizeImg[0]/nbins)**2 # kpc^2

        h2d_faceon /= px_area
        h2d_edgeon /= px_area

        h2d_faceon[h2d_faceon > 0] = np.log10(h2d_faceon[h2d_faceon > 0])
        h2d_edgeon[h2d_edgeon > 0] = np.log10(h2d_edgeon[h2d_edgeon > 0])

        # plot
        fig, (ax_left, ax_right) = plt.subplots(nrows=1, ncols=2, figsize=(13,8), dpi=300)

        ax_left.set_xlabel('x [pc]')
        ax_left.set_ylabel('y [pc]')
        ax_left.set_xlim(size_faceon)
        ax_left.set_ylim(size_faceon)

        extent = [0,u['BoxSize'],0,u['BoxSize']]
        im_left = ax_left.imshow(h2d_faceon, cmap='inferno', extent=extent, aspect=1.0, vmin=vmm[0], vmax=vmm[1])

        ax_right.set_xlabel('z [pc]')
        ax_right.set_ylabel('x [pc]')
        ax_right.set_xlim(size_edgeon)
        ax_right.set_ylim(size_faceon) # should match above

        im_right = ax_right.imshow(h2d_edgeon, cmap='inferno', extent=extent, aspect=1.0, vmin=vmm[0], vmax=vmm[1])

        # colorbar and finish plot
        cax = make_axes_locatable(ax_right).append_axes('right', size='10%', pad=0.1)
        cb = plt.colorbar(im_right, cax=cax)
        cb.ax.set_ylabel('Gas Surface Mass Density [ $\\rm{M_\odot kpc^{-2}}$ ]')

        fig.savefig(filename)
        plt.close(fig)


In [4]:
overrideOldFrames = False

path = "/vera/ptmp/gc/xboecker/run/4_galaxy/xeno_diskIC/run_diskIC_wdm/output_100x_flatMomentumSN/"
savePath = "/vera/u/xboecker/jupyterNotebooksOutputs/plots/Galaxy/SFR_Cooling_XenoSN/movies/100x_FlatMomentumInjection/"
visualize_frames()

#path = "/vera/ptmp/gc/xboecker/run/5_SN_cluster_final/1_density_100/output/"
#savePath = "/vera/u/xboecker/jupyterNotebooksOutputs/plots/Cluster/movies/1_density_100/fixedCbar/"
#visualize_frames_cluster(vmm = [2, 2.6])

numpart_vs_time()
partmass_vs_time()

# ffmpeg -f image2 -start_number 0 -i frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -crf 19 -an -threads 0 out.mp4

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189


In [15]:
path = "/vera/ptmp/gc/xboecker/run/4_galaxy/xeno_diskIC/run_diskIC_wdm/output_100x_starLifeTime_random_0_to_10Myr_zOutflowBoundaries/"
savePath = "/vera/u/xboecker/jupyterNotebooksOutputs/plots/Galaxy/SFR_Cooling_XenoSN/movies/100xRes_zOutflow/highRes/"

path = "/vera/ptmp/gc/xboecker/run/5_SN_cluster_final/1_density_0_2/output/"
savePath = "/vera/u/xboecker/jupyterNotebooksOutputs/plots/Cluster/movies/1_density_0_2/fixedCbar/"

overrideOldFrames = True
#visualize_frames()
visualize_frames_cluster(vmm = [-1, .2])
#numpart_vs_time()
#partmass_vs_time()

# ffmpeg -f image2 -start_number 0 -i frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -crf 19 -an -threads 0 out.mp4

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27